#Classificação automatizada de tomates

Este projeto tem como base a criaçãod e uma classificação automatica de tomates verdes e maduros com a utilização da linguagem de programação python e suas bibliotecas.

##1º passo - Criar um script para buscar imagens na internet
Nesta etapa fiz uma busca através do script de forma separada por fotos de tomates verdes e tomates maduros
Para tal pesquisa foi utilizado a biblioteca python bing_imagem_downloader

In [1]:
from bing_image_downloader import downloader

search = 'tomate maduro'
search2 = 'tomate verde'

downloader.download(search, limit=500,  output_dir='dataset', adult_filter_off=True, force_replace=False, verbose=True)
downloader.download(search2, limit=500,  output_dir='dataset', adult_filter_off=True, force_replace=False, verbose=True)

[%] Downloading Images to e:\ANTONIO\PYTHON\automatizacao_classificacao_tomate\dataset\tomate maduro


[!!]Indexing page: 1

[%] Indexed 35 Images on Page 1.


[%] Downloading Image #1 from https://s3.dia.es/medias/h35/h3e/8834226061342.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://3.bp.blogspot.com/-zrEYCmthErI/TpNP29en6hI/AAAAAAAAHsQ/UWKUt4e-9n4/s1600/9.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://pimpamfruit.com/210-thickbox_default/tomate-maduro.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://biozaki.org/wp-content/uploads/tomate-maduro-biozaki.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://blog.mfrural.com.br/wp-content/uploads/2020/05/tomate-1024x654.jpg
[%] File Downloaded !

[%] Downloading Image #6 from https://fruitagash.es/372-large_default/tomate-maduro-super-oferta.jpg
[%] File Downloaded !

[%] Downloading Image #7 from https://i.ytimg.com/vi/EFWP46evlvk/maxresdefault.jpg
[%] File Downloaded !

[%] 

##2º passo - Pré-processamento

O pré-processamento de dados é uma etapa crucial no desenvolvimento de modelos de aprendizado de máquina, especialmente quando se trabalha com imagens.

 - Redimensionamento: Normalmente, é útil redimensionar todas as imagens para um tamanho padrão. Isso ajuda a garantir que todas as imagens tenham as mesmas dimensões, o que é importante para o treinamento eficaz do modelo. 


In [2]:
import os
from PIL import Image

def redimensionar_imagem(caminho_imagem, novo_tamanho):
    #abrir a imagem usando PIL
    imagem = Image.open(caminho_imagem)
    
    #redimensionar imagens
    imagem_redimensionada = imagem.resize(novo_tamanho)
    
    return imagem_redimensionada

def redimensionar_imagens_em_pasta(pasta_origem, novo_tamanho):
    #percorre todas as imagens na pasta de origem
    for nome_imagem in os.listdir(pasta_origem):
        caminho_imagem_origem = os.path.join(pasta_origem, nome_imagem)
        
        try:
            #tenta abrir a imagem
            imagem_redimensionada = redimensionar_imagem(caminho_imagem_origem, novo_tamanho)
        except Exception as e:
            print(f"Erro ao processar a imagem {caminho_imagem_origem}: {e}")
            continue
        
        #redimensiona a imagem
        imagem_redimensionada = redimensionar_imagem(caminho_imagem_origem, novo_tamanho)
        
        #converte a imagem para o modo RGB se necessário
        if imagem_redimensionada.mode =='RGBA':
            imagem_redimensionada = imagem_redimensionada.convert('RGB')
        
        #substituir imagem original pela nova imagem
        imagem_redimensionada.save(caminho_imagem_origem)

#caminho_imagem = r"E:/ANTONIO/PYTHON/automatizacao_classificacao_tomate/dataset/tomate verde"
caminho_imagem = r"E:/ANTONIO/PYTHON/automatizacao_classificacao_tomate/dataset/tomate maduro"
novo_tamanho = (200, 200)

redimensionar_imagens_em_pasta(caminho_imagem, novo_tamanho)

Normalização: A normalização é outra etapa importante. Ela envolve escalar os valores dos pixels para que estejam dentro de uma faixa específica, como [0, 1] ou [-1, 1]. Isso ajuda a garantir que as características das imagens tenham uma escala consistente e facilita o treinamento do modelo. Você pode normalizar os pixels dividindo-os pelo valor máximo de intensidade (geralmente 255 para imagens RGB).

Augmentação de Dados: Aumentar artificialmente o conjunto de dados pode ajudar a melhorar a capacidade de generalização do modelo e a evitar o overfitting. Isso pode incluir rotações, reflexões, deslocamentos, zooms, entre outros. A biblioteca ImageDataGenerator do TensorFlow ou o módulo ImageDataGenerator do Keras podem ser úteis para aplicar técnicas de aumento de dados.

Conversão para Matriz Numérica: As imagens normalmente são armazenadas como arquivos de imagem, mas para treinar modelos de aprendizado de máquina, elas precisam ser convertidas em matrizes numéricas. Isso pode ser feito usando bibliotecas como OpenCV ou PIL.

Remoção de Ruído: Dependendo da qualidade das imagens, pode ser necessário aplicar técnicas de remoção de ruído para melhorar a qualidade dos dados. Isso pode incluir filtros de suavização, como o filtro de média ou o filtro Gaussiano.

Equalização de Histograma: Em algumas situações, a equalização de histograma pode ser útil para melhorar o contraste das imagens. Isso pode ser especialmente útil se houver variações significativas na iluminação entre as imagens.

Essas são apenas algumas das técnicas comuns de pré-processamento de dados para imagens. A escolha das técnicas a serem aplicadas dependerá do problema específico que você está enfrentando e da natureza dos seus dados. Experimente diferentes técnicas e veja quais funcionam melhor para o seu caso.